In [1]:
import sys
sys.path.append("src")
%load_ext autoreload
%autoreload 2
import pandas as pd
from scraping import scan_swedish_legislation_parallel
from vector_db import VectorDatabaseWeaviate
from embedd import split_text, EmbedderOpenAI, EmbedderSwedishLegislation
from search import SwedishLegislationSearch

## Scrap Swedish government office for all laws and regulations

In [2]:
df = scan_swedish_legislation_parallel()
df.to_pickle("data/laws.pickle")

100%|███████████████████████████████████████| 4539/4539 [01:42<00:00, 44.37it/s]


In [3]:
df = pd.read_pickle("data/laws.pickle")

## Set up embedding

In [2]:
df = pd.read_pickle("data/laws.pickle")

if 0: # For debuging.. 
    df = pd.read_pickle("data/laws.pickle").sample(50)

embedder = EmbedderSwedishLegislation(
    df=df,
    embedd_column="content",
    text_max_length=250,
    text_overlap=20,
    context_columns=["title"],
)
embedder.setup()

In [51]:
r = embedder.embedd_batch(texts=["hejsan", "hejsan på dig"])

In [37]:
embedder._df

,SFS_number,title,issuer,issued_date,in_effect_date,content,n_tokens
0,2023:196,Lag (2023:196) om kommuners ansvar för brottsf...,Departement:Justitiedepartementet L5,2023-04-20,Ikraft:2023-07-01 överg.best.,Lag (2023:196) om kommuners ansvar för brottsf...,115
1,2023:196,Lag (2023:196) om kommuners ansvar för brottsf...,Departement:Justitiedepartementet L5,2023-04-20,Ikraft:2023-07-01 överg.best.,Lag (2023:196) om kommuners ansvar för brottsf...,111
2,2023:196,Lag (2023:196) om kommuners ansvar för brottsf...,Departement:Justitiedepartementet L5,2023-04-20,Ikraft:2023-07-01 överg.best.,Lag (2023:196) om kommuners ansvar för brottsf...,110
3,2023:196,Lag (2023:196) om kommuners ansvar för brottsf...,Departement:Justitiedepartementet L5,2023-04-20,Ikraft:2023-07-01 överg.best.,Lag (2023:196) om kommuners ansvar för brottsf...,106
4,2023:196,Lag (2023:196) om kommuners ansvar för brottsf...,Departement:Justitiedepartementet L5,2023-04-20,Ikraft:2023-07-01 överg.best.,Lag (2023:196) om kommuners ansvar för brottsf...,111
...,...,...,...,...,...,...,...
38371,1736:0123,Handelsbalk (1736:0123 2),Departement:Justitiedepartementet L2,1736-01-23,Ändring införd:t.o.m. SFS1986:798,Handelsbalk (1736:0123 2)||s. 1) ang. löftesma...,117
38372,1736:0123,Handelsbalk (1736:0123 2),Departement:Justitiedepartementet L2,1736-01-23,Ändring införd:t.o.m. SFS1986:798,"Handelsbalk (1736:0123 2)||örfallotid gjort, ä...",109
38373,1736:0123,Handelsbalk (1736:0123 2),Departement:Justitiedepartementet L2,1736-01-23,Ändring införd:t.o.m. SFS1986:798,"Handelsbalk (1736:0123 2)||undit sig att, en f...",103
38374,1736:0123,Handelsbalk (1736:0123 2),Departement:Justitiedepartementet L2,1736-01-23,Ändring införd:t.o.m. SFS1986:798,Handelsbalk (1736:0123 2)||r förfallotiden eft...,102


In [17]:
embedder._df["content"][54]

'Förordning (2023:168) om statsbidrag för kvinnors och flickors organisering||ket gäller även skulder hos Kronofogdemyndigheten som avser återkrav av statsbidrag som Jämställdhetsmyndigheten har beslutat om. Villkor 10 § Den verksamhet som bidrag lämnas för ska bedrivas i Sverige utan vinstsyfte och tillgodose syf'

In [9]:
embedder._df.n_tokens.sum()

24142

In [18]:
# Calculate the cost for the embedding
(embedder._df.n_tokens.sum() / 1000) * 0.0004

1.5549884

In [3]:
# Run the embedder
embedder.embedd(245000, timeout=1/3000)

16469it [1:57:47,  3.00it/s]

In [7]:
embedder._df.to_pickle("data/processed_short.pickle")

In [39]:
embedder._df = pd.read_pickle("data/processed_short.pickle")

## Set up vector database

In [40]:
class_obj = VectorDatabaseWeaviate.setup_class_object_structure(
    class_name="Swedish_legislation",
    names=[
        "SFS_number", "title", "issuer", "content"
    ],
    data_types=["text"] * 4,
    description="Holds Swedish legislation"
)

vector_db = VectorDatabaseWeaviate(
    name="Swedish_legislation",
    url="http://weaviate:8080",
    schema=class_obj,
)

In [41]:
# Clear the vector datasbase 
if 0:
    vector_db.redo()

In [42]:
# Add the embedding vectors to the vector database
vector_db.populate(
    embedder=embedder,
    batch_size=100,
)

38376it [00:52, 734.44it/s]


## Set up the semantic search with chat interface

In [43]:
from search import SwedishLegislationSearch

In [44]:
law_search = SwedishLegislationSearch(
    embedd_function=EmbedderSwedishLegislation.embedd_single,
    vector_db=vector_db,
)

In [54]:
return_data = law_search.query("Vilket ansvar har polisen under ett krigstillstånd? Svara detaljerat", limit=25, print_debug=True)

###########CONTEXT###########

Lag (1943:881) om polisens ställning under krig||1 § En polisman är skyldig att under krig delta i rikets försvar i den omfattning regeringen föreskriver. Lag (2014:581). 2 § En polisman, som enligt särskilda bestämmelser ska delta i rikets försvar, tillhör under krig Försvarsmakten. Lag (20
Kungörelse (1958:262) om tillämpning av lagen (1943:881) om polisens ställning under krig||an endast om polismän redan fullgör nödvändiga civila försvarsuppgifter eller medverkan allvarligt skulle äventyra den allmänna ordningen och säkerheten. När den begärda uppgiften har slutförts får Försvarsmakten inte förfoga över polismännen 
Kungörelse (1958:262) om tillämpning av lagen (1943:881) om polisens ställning under krig||kten omedelbart underrättas. Förordning (2014:1110). 6 § Om polisiär medverkan är nödvändig för att avvärja ett fientligt angrepp, får Försvarsmakten begära medverkan av Polismyndigheten. Myndigheten får avslå en sådan begäran endast om polism
Kungör

In [47]:
return_data = law_search.query("Hur ser processen för ett giftemål ut? Svara detaljerat", limit=25, print_debug=True)

###########CONTEXT###########

Förordning (2009:263) om vigsel som förrättas av särskilt förordnad vigselförrättare||ar: Ja.) Vill du N.N. ta dennadenne N.N. till din hustruman att älska hennehonom i nöd och lust? (Svar: Ja.) (Paret kan växla ringar.) Jag förklarar er nu för äkta makar. När ni nu går ut i livet och åter till vardagen så minns den
Förordning (2009:263) om vigsel som förrättas av särskilt förordnad vigselförrättare|| vigsel bestäms av vigselförrättaren efter samråd med paret. 4 § Vid vigsel ska vigselförrättaren rikta följande ord till paret: Ni har förklarat att ni vill ingå äktenskap med varandra. Vill du N.N. ta dennadenne N.N. till din hustruman?
Förordning (2009:263) om vigsel som förrättas av särskilt förordnad vigselförrättare||till din hustruman? (Svar: Ja.) Vill du N.N. ta dennadenne N.N. till din hustruman? (Svar: Ja.) Jag förklarar er nu för äkta makar. 5 § Om paret särskilt begär det kan vigselförrättaren, i stället för det som anges i 4 §, rikta föl
Kungörel

In [53]:
return_data = law_search.query("Vem kan bli medborgare i Sverige? Svara detaljerat", limit=25, print_debug=True)

###########CONTEXT###########

Lag (1981:324) om medborgarvittnen||ldrabalken. Den som inte är svensk medborgare är valbar endast om han eller hon sedan tre år är folkbokförd i Sverige. Den som är anställd av Polismyndigheten eller Säkerhetspolisen får inte väljas till medborgarvittne. Om ett medborgarvittne 
Passlag (1978:302)||eningen, och 2. för den som uppehåller sig utomlands och inte kan styrka sitt svenska medborgarskap eller har förlorat svenskt medborgarskap utan att förvärva medborgarskap i en annan stat. Lag (2016:135). 4 § Svensk medborgare har rätt
Förordning (2012:694) om svenskt medborgarskap för vissa myndighetschefer||Övrigt:Rättelseblad 2014:1573 har iakttagits.
Lag (2022:700) om särskild kontroll av vissa utlänningar||den som är skäligen misstänkt ska då i stället avse utlänningen. Migrationsverket får fotografera och ta fingeravtryck 28 § När en utlännings rätt att vistas i Sverige handläggs enligt denna lag får Migrationsverket ta fingeravtryck av och 
Lag (2022:70

In [52]:
return_data = law_search.query("Kommer bensinbilar att förbjudas i Sverige? Svara detaljerat", limit=25, print_debug=True)

###########CONTEXT###########

Förordning (2022:315) om miljökrav vid upphandling av bilar och vissa tjänster inom vägtransportområdet||ndirekt ändrad markanvändning, eller - biodrivmedel, syntetiska bränslen eller paraffiniska bränslen som blandas med konventionella fossila bränslen.
Förordning (2022:315) om miljökrav vid upphandling av bilar och vissa tjänster inom vägtransportområdet||veras enligt kommissionens delegerade förordning (EU) 2019807 av den 13 mars 2019 om komplettering av Europaparlamentets och rådets direktiv (EU) 20182001 vad gäller fastställande av bränsleråvaror med hög risk för indirekt ändring av marka
Förordning (2023:132) om producentansvar för bilar||otiverat med hänsyn till befolkningstätheten eller geografiska förutsättningar. En dispens får dock inte ges som innebär att fler än 15 procent av landets bilägare får en längre väg till mottagningsstället än 50 kilometer. En dispens ska 
Förordning (2022:315) om miljökrav vid upphandling av bilar och vissa tjänste